1. Загрузите датасет по ссылке: 
    http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip
    (описание датасета можно посмотреть здесь)
2. Считайте датасет в Python (можете сразу грузить все в память, выборка 
    небольшая), выясните, что используется в качестве разделителей и как 
    проставляются метки классов.
3. Подготовьте для дальнейшей работы два списка: список текстов в порядке 
    их следования в датасете и список соответствующих им меток классов. 
    В качестве метки класса используйте 1 для спама и 0 для "не спама"

In [1]:
import pandas as pd
import numpy as np
data_set = pd.read_csv( "SMSSpamCollection", delimiter="\t", names=( "is_spam", "text" ) )
data_set.is_spam.replace( to_replace=dict( spam = 1, ham = 0 ), inplace = True )

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными 
    настройками, получите из списка текстов матрицу признаков X.
5. Оцените качество классификации текстов с помощью LogisticRegression() с 
    параметрами по умолчанию, используя 
    sklearn.cross_validation.cross_val_score и посчитав среднее 
    арифметическое качества на отдельных fold'ах. Параметр cv задайте 
    равным 10. В качестве метрики качества используйте f1-меру. 
    Получившееся качество – ответ в этом пункте.
6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его 
    помощью класс для следующих сообщений:
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use 
    from your phone now! Subscribe6GB"
    "FreeMsg: Txt: claim your reward of 3 hours talk time"
    "Have you visited the last lecture on physics?"
    "Have you visited the last lecture on physics? Just buy this book and you will have 
    all materials! Only 99\$"
    "Only 99$"
    Выпишите через пробел прогнозы классификатора (0 – не спам, 1 – спам)

In [2]:
def process_data_set( data_set, vectorizer_class, my_ngram_range, classifier_class, test_texts ):
    print( "Processing data using vectorizer " + str( vectorizer_class ) + " with params " + str( my_ngram_range ) + " and classifier " + str( classifier_class ), end=".\n" )
    vectorizer = vectorizer_class( ngram_range = my_ngram_range )
    X = vectorizer.fit_transform( data_set.text )
    classifier = classifier_class()
    scores = cross_val_score( classifier, X, data_set.is_spam, scoring="f1", cv=10 )
    print( "\t10-fold score:", round( np.mean( scores ), 2 ) )
    
    classifier.fit( X, data_set.is_spam )
    print( "\tControl test results:", " ".join( [ str(answer) for answer in classifier.predict( vectorizer.transform( test_texts ) )] ) )

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем 
    ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях 
    измерьте получившееся в кросс-валидации значение f1-меры, 
    округлите до второго знака после точки, и выпишете результаты через 
    пробел в том же порядке. В данном эксперименте мы пробовали 
    добавлять в признаки n-граммы для разных диапазонов n - только 
    биграммы, только триграммы, и, наконец, все вместе - униграммы, 
    биграммы и триграммы. Обратите внимание, что статистики по 
    биграммам и триграммам намного меньше, поэтому классификатор 
    только на них работает хуже. В то же время это не ухудшает результат 
    сколько-нибудь существенно, если добавлять их вместе с 
    униграммами, т.к. за счет регуляризации линейный классификатор не 
    склонен сильно переобучаться на этих признаках.
8. Повторите аналогичный п.7 эксперимент, используя вместо 
    логистической регрессии MultinomialNB(). Обратите внимание, 
    насколько сильнее (по сравнению с линейным классификатором) 
    наивный Байес страдает от нехватки статистики по биграммам и 
    триграммам.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
ranges = [( 1, 1 ), ( 1, 3 ), ( 2, 2 ), ( 3, 3 )]
test_texts = [ "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                 "FreeMsg: Txt: claim your reward of 3 hours talk time",
                 "Have you visited the last lecture on physics?",
                 "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                 "Only 99$"]
for single_range in ranges:
    process_data_set( data_set, CountVectorizer, single_range, LogisticRegression, test_texts )
for single_range in ranges:
    process_data_set( data_set, CountVectorizer, single_range, MultinomialNB, test_texts )

Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 1) and classifier <class 'sklearn.linear_model.logistic.LogisticRegression'>.
	10-fold score: 0.93
	Control test results: 1 1 0 0 0
Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 3) and classifier <class 'sklearn.linear_model.logistic.LogisticRegression'>.
	10-fold score: 0.93
	Control test results: 1 0 0 0 0
Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (2, 2) and classifier <class 'sklearn.linear_model.logistic.LogisticRegression'>.
	10-fold score: 0.82
	Control test results: 1 0 0 0 0
Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (3, 3) and classifier <class 'sklearn.linear_model.logistic.LogisticRegression'>.
	10-fold score: 0.73
	Control test results: 1 0 0 0 0
Processing data using vectorizer <class 'skl

9. Попробуйте использовать в логистической регрессии в качестве 
    признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или 
    понизилось качество на кросс-валидации по сравнению с 
    CountVectorizer на униграммах? Обратите внимание, что результат 
    перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то 
    явление на одном датасете, не надо сразу же его обобщать на любые 
    данные.
#### Качество понизилось: 0.93 против 0.85.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
process_data_set( data_set, TfidfVectorizer, ( 1, 1 ), LogisticRegression, test_texts )

Processing data using vectorizer <class 'sklearn.feature_extraction.text.TfidfVectorizer'> with params (1, 1) and classifier <class 'sklearn.linear_model.logistic.LogisticRegression'>.
	10-fold score: 0.85
	Control test results: 1 1 0 0 0


10*. Попробуйте получить как можно более высокое качество на кросс-
    валидации. Напишите, что пробовали и какое качество получилось.

#### План победы:
##### 1. Честно перебираем тройки ( параметры числа нграм, векторайзер, классификатор ).
##### 2. Осмотрев результаты попробуем поиграть с параметрами классификаторов из самых успешных троек.

In [8]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
max_ngrames = 4
all_ranges = [(i, j) for i in range(1, max_ngrames) for j in range(i, max_ngrames) ]
vectorizers = [CountVectorizer, TfidfVectorizer]
classifiers = [LogisticRegression, MultinomialNB, LinearSVC, RandomForestClassifier, KNeighborsClassifier, AdaBoostClassifier]
for single_range in all_ranges:
    for vectorizer in vectorizers:
        for classifier in classifiers:
            process_data_set( data_set, vectorizer, single_range, classifier, test_texts )

Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 1) and classifier <class 'sklearn.linear_model.logistic.LogisticRegression'>.
	10-fold score: 0.93
	Control test results: 1 1 0 0 0
Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 1) and classifier <class 'sklearn.naive_bayes.MultinomialNB'>.
	10-fold score: 0.93
	Control test results: 1 1 0 0 1
Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 1) and classifier <class 'sklearn.svm.classes.LinearSVC'>.
	10-fold score: 0.94
	Control test results: 1 1 0 0 0
Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 1) and classifier <class 'sklearn.ensemble.forest.RandomForestClassifier'>.
	10-fold score: 0.88
	Control test results: 1 1 0 0 0
Processing data using vectorizer <class 'sklearn.feature_extraction.text.CountVe

Бездарно потратив вычислительные ресурсы, останавливаемся на следующих тройках:

    1) vectorizer <class 'sklearn.feature_extraction.text.TfidfVectorizer'> with params (1, 1) and classifier <class 'sklearn.svm.classes.LinearSVC'> с качеством 0.94.
    2) vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 1) and classifier <class 'sklearn.svm.classes.LinearSVC'> с качеством 0.94
    3) vectorizer <class 'sklearn.feature_extraction.text.CountVectorizer'> with params (1, 1) and classifier <class 'sklearn.naive_bayes.MultinomialNB'> с качеством 0.93, но опознавшим сообщение "Only 99$" из тестового набора за спам.
Попробуем что-нибудь выжать из них меняя параметры классификаторов.

In [14]:
def find_f1_cross_val_score( data_set, vectorizer, classifier ):
    X = vectorizer.fit_transform( data_set.text )
    scores = cross_val_score( classifier, X, data_set.is_spam, scoring="f1", cv=10 )
    return( round( np.mean( scores ), 4 ) )

In [20]:
vectorizers = [CountVectorizer(), TfidfVectorizer()]
penalties = [0.01, 0.1, 0.5, 1, 5, 10, 100, 200]
loss_functions = ["hinge", "squared_hinge"]
tolerances = [1e-2, 1e-3, 1e-4]
max_iterations = [1e2, 1e3, 0.5e4]
best_params = None
best_score = 0

for vectorizer in vectorizers:
    for penalty in penalties:
        for loss_function in loss_functions:
            for tolerance in tolerances:
                for max_iteration in max_iterations:
                    classifier = LinearSVC( C = penalty, loss = loss_function, tol = tolerance, max_iter = max_iteration )
                    current_score = find_f1_cross_val_score( data_set, vectorizer, classifier )
                    if current_score > best_score:
                        best_score = current_score
                        best_params = classifier.get_params()

C:\Users\Nikolay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Nikolay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Nikolay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Nikolay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Nikolay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113:

In [22]:
print( "Best score:", best_score, ". Best params:", best_params, end = '.' )

Best score: 0.9427 . Best params: {'C': 0.1, 'class_weight': None, 'dual': True, 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 100.0, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': None, 'tol': 0.01, 'verbose': 0}.

In [27]:
alphas = [1e-1, 25e-2, 5e-1, 75e-2, 1]
fit_priors = [True, False]
best_score = 0
best_params = None
for alpha in alphas:
    for fit_prior in fit_priors:
        classifier = MultinomialNB( alpha = alpha, fit_prior = fit_prior )
        current_score = find_f1_cross_val_score( data_set, CountVectorizer(), classifier )
        if current_score > best_score:
            best_score = current_score
            best_params = classifier.get_params()

In [28]:
print( "Best score:", best_score, ". Best params:", best_params, end = '.' )

Best score: 0.9376 . Best params: {'alpha': 0.1, 'class_prior': None, 'fit_prior': True}.

#### Подбор параметров классификаторов ситуацию практически не улучшил. Выше ~0.94 прыгнуть не удалось. Продуктовые полученной качество не назвать, но с учетом потраченных сил вполне приемлимым можно. На этом, не имея ориентира, было решено остановиться. :)

11.Какие наблюдения и выводы можно сделать из этого задания?

- Увеличение числа и длины н-грамм не всегда положительно сказывается на качестве. При том эффект в значительной степени зависит от классификатора(что в принципе и ожидаемо).
- Байесовский классификатор сильнее зависит от настройки используеммых н-грамм чем логистическая регрессия.
- Предложенная выборка страдает от недостака спам-текстов. Это легко наблюдать как в соотношении текстов в самой выборке, так и на результатах классификации предложенных в пункте 6 текстов. Из 4-ех предложенных спам-текстов большинство обученных классификаторов правильно классифицирует лишь 1-2, что, мягко говоря, печально.
- Перебор параметров как для векторайзера так и для классификатора практически не дает прироста в качестве в сравнении с дефолтными параметрами. Полагаю, что это также связано со значительной неполнотой выборки в смысле спам-писем.